**CRIANDO ENSEMBLES:Stacking**

Ensembles são conjuntos de modelos que oferecem uma performance melhor
do que cada modelo que o compõe.

In [1]:
#Importando as bibliotecas para carregar os dados

import pandas as pd
import numpy as np

Os dados utilizados são transações comerciais de imóveis na cidade de Ames,
Iowa. O meu objetivo é prever o preço de venda de uma casa na cidade alimentando o modelo com as features.

A base de dados foi retirada de: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/

In [2]:
train=pd.read_csv("train.csv", index_col='Id')
test=pd.read_csv("test.csv", index_col='Id')

#Dividindo a variável de treino em variável resposta e preditiva.

X,y=train.drop("SalePrice",axis=1),train.SalePrice.copy()

In [9]:
test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [15]:
train.head().iloc[:,:5]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street
Id,,,,,
1,60,RL,65.0,8450,Pave
2,20,RL,80.0,9600,Pave
3,60,RL,68.0,11250,Pave
4,70,RL,60.0,9550,Pave
5,60,RL,84.0,14260,Pave


In [18]:
y.head()

Id
1    208500
2    181500
3    223500
4    140000
5    250000
Name: SalePrice, dtype: int64

-Features: Uma das maneiras de obter modelos diferentes é variar a representação dos dados utilizados para treina-los. Assim sendo nosso primeiro passo será construir essa representação.

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

**Funções Auxiliares**

-A métrica para avaliar os modelos é o RMSLE, este erro leva em conta a diferença entre o logaritmo das previsões e do alvo. É possível pensar neste erro como uma aproximação do erro percentual do modelo, mas com propriedades mais interessantes do ponto de vista matemático. Foram criadas 3 funções para o calculo do erro, será feita as tranformações na variável y.

-Usando algumas funções do Scikit-learn que vão ajudar a manter o código mais limpo e mais eficiente, precisamos criar as funções de erro da maneira requerida pelo módulo. Neste caso, a função pecisa receber um modelo treinado, as features e o feature alvo. Ela computará as previsões e deverá retornar um número, que é o valor da métrica de erro.

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.ensemble import RandomForestRegressor

In [4]:
#Criando as 3 funções para o calculo do erro

def rmsle(estimator, X, y):
    p = estimator.predict(X)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))#log do y e log da previsão.

def rmsle_log_y(estimator, X, y):
    p = estimator.predict(X)
    return np.sqrt(mean_squared_error(y, p))

def rmsle_sqrt_y(estimator, X, y):
    p = estimator.predict(X)
    y = np.power(y, 2)#Valores elevados para a segunda potência.
    p = np.power(p, 2)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))

In [5]:
#Devido a função do scikitlearn requerir os dados treinados, é feito uma validação cruzada.

kf = KFold(n_splits=5, shuffle=True, random_state=1)

**Feature set 1:** Numéricas

Conjunto com as variáveis originalmente numéricas, tem que selecionar todas as colunas que possuem números int ou float.
Para substituir os valores numlos iremos utilizar o fillna(-1).
Com os dados selecionados iremos treinar o modelo por meio do Random Forest dentro da validação cruzada. Esta função retorna uma lista com os erros de cada iteração da validação cruzada, então fiz a média para sabermos o erro médio das partes.

In [6]:
X1 = X.select_dtypes(include=[np.number]).fillna(-1)

In [30]:
X1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   LotFrontage    1460 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   OverallQual    1460 non-null   int64  
 4   OverallCond    1460 non-null   int64  
 5   YearBuilt      1460 non-null   int64  
 6   YearRemodAdd   1460 non-null   int64  
 7   MasVnrArea     1460 non-null   float64
 8   BsmtFinSF1     1460 non-null   int64  
 9   BsmtFinSF2     1460 non-null   int64  
 10  BsmtUnfSF      1460 non-null   int64  
 11  TotalBsmtSF    1460 non-null   int64  
 12  1stFlrSF       1460 non-null   int64  
 13  2ndFlrSF       1460 non-null   int64  
 14  LowQualFinSF   1460 non-null   int64  
 15  GrLivArea      1460 non-null   int64  
 16  BsmtFullBath   1460 non-null   int64  
 17  BsmtHalfBath   1460 non-null   int64  
 18  FullBath

In [7]:
print('Dims', X1.shape)

Dims (1460, 36)


In [28]:
#Rodando o modelo utilizado

model = RandomForestRegressor(n_estimators=1000, random_state=0)

#Fazendo a cross validation

error = cross_val_score(model, X1, y, cv=kf,scoring=rmsle)

In [29]:
#Indicando o valor do erro:

print('RMSLE:', error.mean())

RMSLE: 0.14582352617986846


**Feature set 1:**Categóricas

Agora as variáveis categóricas serão adicionadas ao conjunto de features. Um dos modos de fazer estas tranformações uma delas é substituir cada valor por números sequenciais. Em alguns modelos isso pode ser problemático, pois eles tentarão capturar alguma relação de ordem em valores que podem não ter. No nosso caso, com modelos baseados em árvores de decisão, este problema é quase inexistente. Irá ser feita novamente a validação cruzada agora com os nvos dados.

In [13]:
from sklearn.preprocessing import LabelEncoder

In [16]:
X2.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [18]:
# Método Ordinal
# Fazendo a tranformação por meio da função for.

X2 = X.copy()
for col in X2.columns:
    if X2[col].dtype == object:
        enc = LabelEncoder()
        X2[col] = enc.fit_transform(X[col].fillna('Missing'))



In [19]:
X2

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,3,65.0,8450,1,1,3,3,0,4,...,0,0,3,2,1,0,2,2008,8,4
2,20,3,80.0,9600,1,1,3,3,0,2,...,0,0,3,2,1,0,5,2007,8,4
3,60,3,68.0,11250,1,1,0,3,0,4,...,0,0,3,2,1,0,9,2008,8,4
4,70,3,60.0,9550,1,1,0,3,0,0,...,0,0,3,2,1,0,2,2006,8,0
5,60,3,84.0,14260,1,1,0,3,0,2,...,0,0,3,2,1,0,12,2008,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,3,62.0,7917,1,1,3,3,0,4,...,0,0,3,2,1,0,8,2007,8,4
1457,20,3,85.0,13175,1,1,3,3,0,4,...,0,0,3,3,1,0,2,2010,8,4
1458,70,3,66.0,9042,1,1,3,3,0,4,...,0,0,3,0,3,2500,5,2010,8,4


In [20]:
print('Dims', X2.shape)

X2.fillna(-1, inplace=True)
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X2, y, cv=kf,scoring=rmsle).mean()

print('RMSLE:', error)

Dims (1460, 79)
RMSLE: 0.14383736485915208


OneHot Encoding: Esee método também pode ser usada para a transformação das variáveis categóricas, este método basicamente consiste em transformar cada valor da variável em uma coluna cujo novo valor será 1 caso a variável tenha aquele valor num determinado exemplo, ou 0 em caso negativo. Existe indicativos que este processo não ocorre tao bem em arvores de decisão. Este processo cria mais de 200 colunas adicionais portanto apenas irei indica-lo como uma alternativa.

In [22]:
from sklearn.preprocessing import OneHotEncoder

X3 = X.copy()
cats = []
for col in X3.columns:
    if X3[col].dtype == object:
        X3 = X3.join(pd.get_dummies(X3[col], prefix=col),how='left')
        X3.drop(col, axis=1, inplace=True)

In [23]:
X3

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
1457,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,0,0,1,0,0,0,0,1,0
1458,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,0,0,1,0,0,0,0,1,0


In [25]:
print('Dims', X3.shape)

X3.fillna(-1, inplace=True)
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X3, y, cv=kf,scoring=rmsle).mean()

print('RMSLE:', error)

Dims (1460, 288)
RMSLE: 0.14399551277366124


**Transformando a variável Alvo**: Uma maneira de obter outros métodos para comparação do melhor resultado, é tranformar as variáveis numéricas. No caso vou utilizar logaritmo e raiz quadrada na variável y, buscando um menor erro para a regressão.

In [30]:
#Utilizando o Log

#Para X1(Apenas as variáveis originalmente numéricas)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X1, np.log1p(y), cv=kf,scoring=rmsle_log_y).mean()
print('RF, X1, log-target RMSLE:', error)

#Para X2(Todas as variáveis com as categoticas tranformadas em numéricas)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X2, np.log1p(y), cv=kf,scoring=rmsle_log_y).mean()
print('RF, X2, log-target RMSLE:', error)

RF, X1, log-target RMSLE: 0.14516026930052636
RF, X2, log-target RMSLE: 0.14209480789088058


In [31]:
#Utilizando a raiz quadrada

#Para X1(Apenas as variáveis originalmente numéricas)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X1, np.sqrt(y), cv=kf,scoring=rmsle_sqrt_y).mean()
print('RF, X1, sqrt-target RMSLE:', error)

#Para X2(Todas as variáveis com as categoticas tranformadas em numéricas)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X2, np.sqrt(y), cv=kf,scoring=rmsle_sqrt_y).mean()
print('RF, X2, sqrt-target RMSLE:', error)

RF, X1, sqrt-target RMSLE: 0.14565293448427205
RF, X2, sqrt-target RMSLE: 0.14300460013198157


Pode ser visto que utilizando as duas tranformações os resultados apresentaram uma pequena melhora. Pode-se sugerir que este efeito esta derivado aos dados (y) estarem mais próximos de uma distribuição normal, facilitando o trabalho do modelo.
    

**Gerenado um novo modelo:** Outra maneira de gerar diversidade para o ensemble é gerar modelos diferentes. No caso irei utilizar o modelo baseado tambem em arvores de decisão, o GBM, este modelo basicamente treina cada árvore sequencialmente focando nos erros cometidos pelas arvores anteriores.

In [34]:
from sklearn.ensemble import GradientBoostingRegressor

#Gerando um novo modelo utilizando os dados X1 e com y utilizando log

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X1, np.log1p(y), cv=kf,scoring=rmsle_log_y).mean()
print('GBM, X1, sqrt-target RMSLE:', error)

#Gerando um novo modelo utilizando os dados X2 e com y utilizando log

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X2, np.log1p(y), cv=kf,scoring=rmsle_log_y).mean()
print('GBM, X2, sqrt-target RMSLE:', error)

GBM, X1, sqrt-target RMSLE: 0.13349245491356668
GBM, X2, sqrt-target RMSLE: 0.12980689048155075


In [35]:
#Gerando um novo modelo utilizando os dados X1 e com y utilizando raiz quadrada

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X1, np.sqrt(y), cv=kf,scoring=rmsle_sqrt_y).mean()
print('GBM, X1, sqrt-target RMSLE:', error)

#Gerando um novo modelo utilizando os dados X2 e com y utilizando raiz quadrada

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X2, np.sqrt(y), cv=kf,scoring=rmsle_sqrt_y).mean()
print('GBM, X2, sqrt-target RMSLE:', error)

GBM, X1, sqrt-target RMSLE: 0.13466955860674396
GBM, X2, sqrt-target RMSLE: 0.13107304131368688


É possivel ver uma melhora deste modelo em relação ao uso do RandomForest, portanto ao utilizar o GBM, ocorre uma maior captura dos padrões das features em relação a variável alvo(Preço do Imóveis).

**Ajuste dos Hiperparâmetros:** Caso seja feito o ajuste do hiperparâmetros eles precisam ser ajustados usando dados separados de validação ou um ciclo de validação cruzada. A questão do Stacking não é sempre a utilização dos melhores modelos mais sim os mais diversos para a captação de caracteristicas diferentes. Caso seja feito o ajuste é importante colocá-lo dentro do ciclo de validação cruzada que veremos no próximo passo.

**Stacking:** O Stacking é uma maneira de fazer o ensemble na qual usamos modelos para fazer previsões, e depois usamos estas previsões como features em novos modelos, no que pode ser chamado de "segundo nível". Você pode fazer este processo
várias vezes, mas a cada nível o retorno em performance com relação à computação necessária é menor.
    -Nesta fase precisamos de dois ciclos de validação cruzada: externo e interno. No interno, nós treinaremos os modelos nos dados originais e faremos as previsões. No externo, treinaremos o modelo usando as previsões do primeiro passo como features.
    -A cada passo da validação cruzada interna, vamos salvar as previsões para a parte dos dados que for usada como validação. Desta maneira teremos previsões para todos os exemplos de nossos dados de treino. Além disso treinaremos um
modelo nos dados originais de treino para podermos fazer previsões para os dados de teste.
    -No ciclo externo treinaremos um modelo nas previsões geradas pelo ciclo interno, e as features dos dados de validação serão as previsões dos modelos de primeiro nível nos dados de teste.
    -No nosso caso específico, criaremos previsões usando todas as combinações de modelos (RF e GBM), transformações do target (log e raiz quadrada) e feature sets (X1 e X2). Não usaremos o X3 porque levaria muito tempo para treinar. No fim teremos previsões de 8 modelos no primeiro nível. No segundo nível usei uma regressão linear regularizada (Ridge). Tendo estas previsões podemos computar o erro nos dados fora das nossas amostras de treino e validação internas, o que nos dará uma estimativa confiável do erro do ensemble.

In [36]:
from itertools import product
from sklearn.linear_model import Ridge

In [46]:
#Fazendo os estimadores internos e externos da validação cruzada.

kf_out = KFold(n_splits=5, shuffle=True, random_state=1)
kf_in = KFold(n_splits=5, shuffle=True, random_state=2)

#Fazendo a estimação

cv_mean = []
for fold, (tr, ts) in enumerate(kf_out.split(X, y)):
    X1_train, X1_test = X1.iloc[tr], X1.iloc[ts]
    X2_train, X2_test = X2.iloc[tr], X2.iloc[ts]
    y_train, y_test = y.iloc[tr], y.iloc[ts]
    
#Colocando os modelos em listas 

    modelos = [GradientBoostingRegressor(random_state=0), RandomForestRegressor(random_state=0)]
    targets = [np.log1p, np.sqrt]
    feature_sets = [(X1_train, X1_test), (X2_train, X2_test)]

#Iniciando o Stacking

    predictions_cv = []
    predictions_test = []
    for model, target, feature_set in product(modelos, targets, feature_sets):
        predictions_cv.append(cross_val_predict(model,feature_set[0], target(y_train), cv=kf_in).reshape(-1,1))
        model.fit(feature_set[0], target(y_train))#treinando o modelo
        ptest = model.predict(feature_set[1])#Fazendo a previsão interna utilizand RF e GBM
        predictions_test.append(ptest.reshape(-1,1))
    
    predictions_cv = np.concatenate(predictions_cv, axis=1)
    predictions_test = np.concatenate(predictions_test, axis=1)

    stacker = Ridge()#Fazendo a previsão dos segundo nivel com a regressão linear(Ridge)
    stacker.fit(predictions_cv, np.log1p(y_train))
    error = rmsle_log_y(stacker, predictions_test, np.log1p(y_test))
    cv_mean.append(error)
    print('RMSLE Fold %d - RMSLE %.4f' % (fold, error))
    
print('RMSLE CV5 %.4f' % np.mean(cv_mean))

RMSLE Fold 0 - RMSLE 0.1251
RMSLE Fold 1 - RMSLE 0.1440
RMSLE Fold 2 - RMSLE 0.1267
RMSLE Fold 3 - RMSLE 0.1378
RMSLE Fold 4 - RMSLE 0.1078
RMSLE CV5 0.1283


Pode-se ver que o melhor modelo no primeiro nível é o GBM treinado com a transformação log nos dados X2, que atinge o erro de 0,1298. Nosso ensemble atinge o valor de 0,1251. Uma melhora de 0,62%.

RMSLE Fold 0 (GBM, X2)
RMSLE Fold 1 (GBM, X1)
RMSLE Fold 2 (RF,  X2)
RMSLE Fold 3 (RF,  X1)
RMSLE Fold 4 (Ridge)
